In [1]:
import google.generativeai as genai
from dotenv import load_dotenv
import os
import requests
from io import BytesIO
import glob
import os

# Load environment variables from the .env file
load_dotenv()

# Retrieve the API key from the environment variable
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)
generation_config = genai.GenerationConfig(
    temperature=0.0,
    response_mime_type="application/json"
)

In [2]:
system_instruction_prompt= """
You are a Cognitive Specialist AI. Your task is to refer to the attached papers, then generate factual, score‑driven descriptions for a “Child Profile” JSON object. Do not invent data or hallucinate—only use established research-based language.  

When prompted, you will receive:  
1. One or more uploaded PDF resource papers.  
2. A JSON skeleton with numeric scores and empty description fields (as in the example below).  

Your job is to populate every `"description"` or summary and also modalities of the kids profile field based strictly on the score values and established cognitive‑domain definitions in the reference material. Output **only** the completed JSON object—no extra commentary, no markdown, and no omitted fields.  

Example input schema:
```json
{
  "name": "Kid",
  "gender": "",
  "age": 1,
  "diagnosis": "not identified",
  "cognitiveDomainProfile": {
    "workingMemory": {
      "score": 72,
      "description": ""
    },
    "processingSpeed": {
      "score": 68,
      "description": ""
    },
    "attentionSpan": {
      "visualAttention": {
        "score": 65,
        "description": ""
      },
      "auditoryAttention": {
        "score": 70,
        "description": ""
      }
    }
  },
  "executiveFunction": {
    "planning": 4,
    "impulseControl": 3,
    "description": ""
  },
  "learningPreferences": {
    "modalities": [],
    "description": ""
  },
  "learningChallengesSummary": {
    "attentionAndFocus": "",
    "memoryAndSpeed": "",
    "planningAndImpulsivity": "",
    "strengths": ""
  }
}


 """

In [3]:
profile1 = {
  "name": "Kid",
  "gender": "male",
  "age": 13,
  "diagnosis": "Not formally diagnosed, but exhibiting early symptoms suggestive of attention deficit hyperactivity disorder (ADHD)",
  "cognitiveDomainProfile": {
    "workingMemory": {
      "score": 85,
      "description": ""
    },
    "processingSpeed": {
      "score": 78,
      "description": ""
    },
    "attentionSpan": {
      "visualAttention": {
        "score": 80,
        "description": ""
      },
      "auditoryAttention": {
        "score": 75,
        "description": ""
      }
    }
  },
  "executiveFunction": {
    "planning": 6,
    "impulseControl": 6,
    "description": ""
  },
  "learningPreferences": {
    "modalities": [],
    "description": ""
  },
  "learningChallengesSummary": {
    "attentionAndFocus": "",
    "memoryAndSpeed": "",
    "planningAndImpulsivity": "",
    "strengths": ""
  }
}


profile2= {
  "name": "Kid",
  "gender": "female",
  "age": 13,
  "diagnosis": "Attention Deficit Hyperactivity Disorder (ADHD)",
  "cognitiveDomainProfile": {
    "workingMemory": {
      "score": 85,
      "description": ""
    },
    "processingSpeed": {
      "score": 78,
      "description": ""
    },
    "attentionSpan": {
      "visualAttention": {
        "score": 80,
        "description": ""
      },
      "auditoryAttention": {
        "score": 75,
        "description": ""
      }
    }
  },
  "executiveFunction": {
    "planning": 6,
    "impulseControl": 6,
    "description": ""
  },
  "learningPreferences": {
    "modalities": [],
    "description": ""
  },
  "learningChallengesSummary": {
    "attentionAndFocus": "",
    "memoryAndSpeed": "",
    "planningAndImpulsivity": "",
    "strengths": ""
  }
}

profile3= {
  "name": "Kid",
  "gender": "female",
  "age": 13,
  "diagnosis": "Identified as Gifted and Talented (IQ > 130)",
  "cognitiveDomainProfile": {
    "workingMemory": {
      "score": 135,
      "description": ""
    },
    "processingSpeed": {
      "score": 140,
      "description": ""
    },
    "attentionSpan": {
      "visualAttention": {
        "score": 145,
        "description": ""
      },
      "auditoryAttention": {
        "score": 142,
        "description": ""
      }
    }
  },
  "executiveFunction": {
    "planning": 10,
    "impulseControl": 9,
    "description": ""
  },
  "learningPreferences": {
    "modalities": [],
    "description": ""
  },
  "learningChallengesSummary": {
    "attentionAndFocus": "",
    "memoryAndSpeed": "",
    "planningAndImpulsivity": "",
    "strengths": ""
  }
}


In [4]:
pattern = os.path.join("papers_profiles", "*.pdf")
pdf_paths = glob.glob(pattern)

print(pdf_paths)

['papers_profiles\\Assessing executive functions in children.pdf', 'papers_profiles\\Assessments Related to the Physical, Affective and Cognitive Domains of Physical Literacy.pdf', 'papers_profiles\\Auditory and Visual Attention in Normal and ADHD-Inattentive subtype Children.pdf', 'papers_profiles\\Domains of cognition and their assessment.pdf', 'papers_profiles\\Evidence-based Assessment of Cognitive Functioning.pdf', 'papers_profiles\\Mapping Normative Trajectories of Cognitive.pdf', 'papers_profiles\\Measurement of Cognition for the.pdf', 'papers_profiles\\wisc-v-interpretive-report.pdf', 'papers_profiles\\wms-iv-interpretive-report-sample-female.pdf']


In [5]:
model = genai.GenerativeModel(
    model_name='models/gemini-1.5-pro',           
    system_instruction=system_instruction_prompt,  
    generation_config=generation_config,
       
)

# pdf_paths = [
#     "papers_profiles\Assessing executive functions in children.pdf",
#     "papers_profiles\Assessments Related to the Physical, Affective and Cognitive Domains of Physical Literacy.pdf",
#     "papers_profiles\Mapping Normative Trajectories of Cognitive.pdf",
#     "papers_profiles\Domains of cognition and their assessment.pdf",
#     "papers_profiles\Evidence-based Assessment of Cognitive Functioning.pdf",
#     "papers_profiles\Measurement of Cognition for the.pdf",
#     "papers_profiles\Auditory and Visual Attention in Normal and ADHD-Inattentive subtype Children.pdf"
# ]


# build the pattern to match all .pdf files in that folder


# 5) Upload each PDF
pdf_files = [
    genai.upload_file(path=path, mime_type="application/pdf")
    for path in pdf_paths
]  # returns a list of File objects :contentReference[oaicite:0]{index=0}

# 6) Ask Gemini to read all three before answering
user_prompt = """
Please read the attached papers and use them to fill in the JSON skeleton below. {kid_profile}
The JSON object should be a complete and accurate representation of the child's cognitive profile including the modalities, based on the provided scores and the information in the papers. Ensure that all fields are populated according to the guidelines in the system instruction prompt.
"""

response = model.generate_content(
    pdf_files + [user_prompt.format(kid_profile=profile1)], 
)

print(response.text)

{"name": "Kid", "gender": "male", "age": 13, "diagnosis": "Not formally diagnosed, but exhibiting early symptoms suggestive of attention deficit hyperactivity disorder (ADHD)", "cognitiveDomainProfile": {"workingMemory": {"score": 85, "description": "Kid's working memory is within the average range. He demonstrates a good ability to register, maintain, and manipulate visual and auditory information in conscious awareness. He can perform tasks requiring attention and concentration, as well as visual and auditory discrimination."}, "processingSpeed": {"score": 78, "description": "Kid's processing speed is within the average range. He demonstrates an average rate at which mental operations are performed. His reaction time to various stimulus arrays is average, and he takes an average amount of time to perform mental activities."}, "attentionSpan": {"visualAttention": {"score": 80, "description": "Kid's visual attention is within the average range. He demonstrates an average ability to mai

In [6]:
response = model.generate_content(
    pdf_files + [user_prompt.format(kid_profile=profile2)], 
)

print(response.text)

{"name": "Kid", "gender": "female", "age": 13, "diagnosis": "Attention Deficit Hyperactivity Disorder (ADHD)", "cognitiveDomainProfile": {"workingMemory": {"score": 85, "description": "Kid's working memory is within the average range for her age, reflecting an age-appropriate ability to register, maintain, and manipulate visual and auditory information in conscious awareness.  She demonstrates adequate attention, concentration, and auditory and visual discrimination skills for her age."}, "processingSpeed": {"score": 78, "description": "Kid's processing speed is within the average range. This suggests an age-appropriate rate at which she performs mental operations, including reaction time to stimuli and time needed to perform mental activities.  Her speed is likely adequate for short-term memory scanning and perceptual comparisons (e.g., same vs. different)."}, "attentionSpan": {"visualAttention": {"score": 80, "description": "Kid's visual attention is within the average range, suggest

In [7]:
response = model.generate_content(
    pdf_files + [user_prompt.format(kid_profile=profile3)], 
)

print(response.text)

{"name": "Kid", "gender": "female", "age": 13, "diagnosis": "Identified as Gifted and Talented (IQ > 130)", "cognitiveDomainProfile": {"workingMemory": {"score": 135, "description": "Kid exhibits an exceptional ability to register, maintain, and manipulate visual and auditory information in conscious awareness, indicative of superior attention, concentration, and mental control.  Her capacity for sequential processing, mental manipulation, and attention are also very strong. She appears to effectively utilize working memory in the service of learning, problem-solving, and higher-order reasoning."}, "processingSpeed": {"score": 140, "description": "Kid demonstrates remarkable speed and accuracy in visual identification, decision-making, and decision implementation.  Her rapid processing of visual information, quick and accurate decisions, and swift implementation of those decisions are evident.  This suggests that processing speed is not a limiting factor in her performance on tasks inv

In [8]:
pdf_files

[genai.File({
     'name': 'files/ggse33j39vy3',
     'display_name': 'Assessing executive functions in children.pdf',
     'mime_type': 'application/pdf',
     'sha256_hash': 'MGRhZGQ3NTUwNGNjODdkNWIwZWVjY2ZlODllZjdjOTI3NjE3ZjFkZTYyZjJhMmZjODlmMjdkYzg4NDdmNWJlZg==',
     'size_bytes': '476328',
     'state': 'ACTIVE',
     'uri': 'https://generativelanguage.googleapis.com/v1beta/files/ggse33j39vy3',
     'create_time': '2025-04-22T18:24:27.976039Z',
     'expiration_time': '2025-04-24T18:24:27.943995064Z',
     'update_time': '2025-04-22T18:24:27.976039Z'}),
 genai.File({
     'name': 'files/1al7km8uxnxd',
     'display_name': 'Assessments Related to the Physical, Affective and Cognitive '
                     'Domains of Physical Literacy.pdf',
     'mime_type': 'application/pdf',
     'sha256_hash': 'ZDY3NzQyODdhMTZlMmY5MmMyZTljYTg0Njk2NzBjM2RmZDdkMjUwMWExNzI0MTQyODhmMTBmNDdjZWVhMmE3Zg==',
     'size_bytes': '1884494',
     'state': 'ACTIVE',
     'uri': 'https://generativelanguage.